In [86]:
'''
This notebook shows the python code and the dresult for the project.
Highlights:
Two different methods were applied here to detect anomly:
1. On account level, if the count, amount, avg_spend deviates more then X times of standard deviation from the mean,
then the current observtion is considered to be an anomaly for this account.
Here avg_spend is a created feature defined as count/amount.
2. For all accounts, min, max, and mean of count and amount were calculated and used as feature for this account. 
Then Kmeans clustering was applied to all accounts. 
Accounts that are in small clusters are highly likely to be anomalies.
'''

'\nThis notebook shows the python code and the dresult for the project.\nHighlights:\nTwo different methods were applied here to detect anomly:\n1. On account level, if the count, amount, avg_spend deviates more then X times of standard deviation from the mean,\nthen the current observtion is considered to be an anomaly for this account.\nHere avg_spend is a created feature defined as count/amount.\n2. For all accounts, min, max, and mean of count and amount were calculated and used as feature for this account. \nThen Kmeans clustering was applied to all accounts. \nAccounts that are in small clusters are highly likely to be anomalies.\n'

In [87]:
import pandas as pd
import numpy as np
from datetime import *
import time
import plotly
from plotly import __version__
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import random
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [88]:
#read in csv files
counts = pd.read_csv("counts.csv")
spend = pd.read_csv("spend.csv")
#browse the dataframes
print (counts.head())
print (spend.head())

                            account        date  count
0  5e0a5eb5575517efdddb2799c90e2e9b  2017-08-16     56
1  a04e063493c55bc6cecd9056712e9e47  2017-08-16    180
2  ff6cc8cadcb7457f9ddb2dd238a13733  2017-08-16    432
3  324a05f1f917b8a0af83a54e55a1ef63  2017-08-16     50
4  b9abff45dc08172b556cb10b7c3c2252  2017-08-16    135
                            account        date       amount
0  5e0a5eb5575517efdddb2799c90e2e9b  1502841600    15.144020
1  a04e063493c55bc6cecd9056712e9e47   102841600   730.070453
2  ff6cc8cadcb7457f9ddb2dd238a13733  1502841600  3651.412248
3  324a05f1f917b8a0af83a54e55a1ef63  1502841600   113.219243
4  b9abff45dc08172b556cb10b7c3c2252  1502841600   697.224210


In [89]:
#Column "date" seems to be epoch time in 'spend'
#It is confirmed after comparing the dates in the two files
#generate epoch time in `counts`  
counts['date_new'] = counts.apply(lambda row: datetime.strptime(row['date'], '%Y-%m-%d'), axis = 1)
                                
counts['epoch'] = counts.apply(lambda row: \
                                (date(int(row['date'][:4]), int(row['date'][5:7]), int(row['date'][8:10])) - date(1970,1,1)).days*24*60*60, \
                                axis = 1)
#merge counts and spend
df_all = pd.merge(counts, spend, left_on = ['account', 'epoch'], right_on = ['account', 'date'], how = 'inner')

In [90]:
#simple check to see if df_all is merged correctly
print (len(counts))
print (len(spend))
print (len(df_all))

1934028
1934135
1934025


In [91]:
##calculate some potentially interesting features
#First feature: average spend per account
df_all['avg_spend'] = df_all.apply(lambda row: row['amount']/row['count'], axis = 1)
#Other interesting features include:
#deviation of the count/amount/avg_spend from mean for each acccount
#change of count/amount/avg_spend compared to the previous day
#Given the time limit, I won't spend time exploring these features

In [92]:
##Question 1: For each account, at which time period does the account shows anomaly behavior from normal?
#First define the statistic anomaly here:
#daily value differs from mean by more than 10 standard deviation
#The metric could be count, amount, or avg_spend
def detect_anomaly(val, val_mean, val_std):
    if abs(val - val_mean) > val_std * 10:
        return True
    else:
         return False
def get_mean(data, variable, metric):
    return data.join(data.groupby([variable])[metric].mean(), on = variable, rsuffix='_mean')
def get_std(data, variable, metric):
    return data.join(data.groupby([variable])[metric].std(), on = variable, rsuffix='_std')
def detect_anomaly_for_metric(data, variable, metric):
    data = get_mean(data, variable, metric)
    data = get_std(data, variable, metric)
    print ("Example plot for anomaly according to %s" %metric)
    data['is_anomalous_%s' % metric]  = data.apply(lambda row: detect_anomaly(row[metric],\
                                                          row['%s_mean' % metric],\
                                                          row['%s_std' % metric]), axis = 1)
    data['color'] = np.where(data['is_anomalous_%s' % metric] == True, 1, 0)
    print ('anomaly distribution for metric %s:' %metric)
    print (data['is_anomalous_%s' % metric].value_counts())
    #plot a random example with anomaly detected
    anomaly_list = data[data['is_anomalous_%s' % metric] == True]['account']
    random_anomaly = random.choice(anomaly_list.tolist())
    plot_data = data[data['account'] == random_anomaly].sort_values(['date_new'])
    trace0 = Scatter(x = plot_data['date_new'] ,
                        y = plot_data[metric] , 
                        mode = 'lines+markers',
                        name = metric,
                        marker = dict(
                        color = plot_data['color']
                        ))
    data = [trace0]
    iplot(data, filename='scatter-mode')
    return data


In [ ]:
#Note that 10 is a randomly picked number. 
#It should be adjusted according to the False positive rate/False negative rate if we know which are real anomaly.
#The threshold here could be different for different metrics.

In [93]:
#run detect_anomaly for all three metrics: count, amount, avg_spend
data = [detect_anomaly_for_metric(df_all, 'account', metric) for metric in ['count', 'amount', 'avg_spend']]

Example plot for anomaly according to count
anomaly distribution for metric count:
False    1934004
True          21
Name: is_anomalous_count, dtype: int64


Example plot for anomaly according to amount
anomaly distribution for metric amount:
False    1931941
True        2084
Name: is_anomalous_amount, dtype: int64


Example plot for anomaly according to avg_spend
anomaly distribution for metric avg_spend:
False    1931488
True        2537
Name: is_anomalous_avg_spend, dtype: int64


In [75]:
#Question 2: Among all observations, what accounts are the outliers?
#Use K-means clustering
#create new features: min, max, and mean of count for each account during the observation period
df_all = df_all.join(df_all.groupby('account', as_index = True)['count'].agg(["max", "min", "mean"]), on = 'account', rsuffix='_count')

In [76]:
df_all = df_all.join(df_all.groupby('account', as_index = True)['count'].agg(["max", "min", "mean"]), on = 'account', rsuffix='_count')
X = df_all[['account', 'max_count', 'min_count', 'mean_count']].drop_duplicates()
X = X[['max_count', 'min_count', 'mean_count']]
kmeans = KMeans(n_clusters = 3, random_state = 0).fit(X)
kmeans.labels_
#observations in small clusters are likely to be outliers
pd.value_counts(pd.Series(kmeans.labels_))

1    9503
0     469
2      27
dtype: int64

In [77]:
trace1 = Scatter3d(x = X['max_count'] ,
                   y = X['min_count'] , 
                   z = X['mean_count'],
                   mode = 'markers',
                   marker = dict(
                   size = 3))
data = [trace1]
layout = Layout(
    scene=Scene(
        xaxis=XAxis(title='max count'),
        yaxis=YAxis(title='min count'),
        zaxis=ZAxis(title='mean count')
    )
)
fig = Figure(data = data, layout = layout)
iplot(fig, filename='scatter-mode-3d')

In [78]:
#create new features: min, max, and mean of amount for each account during the observation period
df_all = df_all.join(df_all.groupby('account', as_index = True)['amount'].agg(["max", "min", "mean"]), \
                     on = 'account',  rsuffix='_amount')

In [79]:
X = df_all[['account', 'max_amount', 'min_amount', 'mean_amount']].drop_duplicates()
X = X[['max_amount', 'min_amount', 'mean_amount']]
kmeans = KMeans(n_clusters = 3, random_state = 0).fit(X)
kmeans.labels_
#observations in small clusters are likely to be outliers
pd.value_counts(pd.Series(kmeans.labels_))

0    9267
2     716
1      16
dtype: int64

In [80]:
trace1 = Scatter3d(x = X['max_amount'] ,
                   y = X['min_amount'] , 
                   z = X['mean_amount'],
                   mode = 'markers',
                   marker = dict(
                   size = 3))
data = [trace1]
layout = Layout(
    scene=Scene(
        xaxis=XAxis(title='max amount'),
        yaxis=YAxis(title='min amount'),
        zaxis=ZAxis(title='mean amount')
    )
)
fig = Figure(data = data, layout = layout)
iplot(fig, filename='scatter-mode-3d')

In [ ]:
#Note: here the number of clusters is arbitrarily chosen. 
#The optimal number of clusters could be chosen using the elbow method.